Importing libraries

In [1]:
import pandas as pd

# Example DataFrame
data = {
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50]
}

df = pd.DataFrame(data)

# Define the condition
condition = df['A'] > 3

# Filter rows based on the condition
new_df = df[condition]

# Save the new DataFrame
new_df.to_csv('filtered_rows.csv', index=False)  # Or save it in any other format

print("Filtered DataFrame:")
print(new_df)


Filtered DataFrame:
   A   B
3  4  40
4  5  50


In [2]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\Additional-Task\\storage_key.json"

In [2]:
d = pd.read_csv("D:\\Work and Assignments\\Python\\demo.csv")
d

,product_name,product_url


Reading csv file containing product_name and url

In [36]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\sampleCSV.csv")
data.shape

(155, 2)

Storing url column in data_url

In [37]:
data_url = data['product_url']
data_url

0      https://m.media-amazon.com/images/I/71xWtLhH2G...
1      https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2      https://m.media-amazon.com/images/I/71YG7EClYf...
3      https://m.media-amazon.com/images/I/81ATe15IyH...
4      https://m.media-amazon.com/images/I/61OBcY37KX...
                             ...                        
150    https://m.media-amazon.com/images/I/41FpPQlmcf...
151    https://m.media-amazon.com/images/I/71BXLqma3B...
152    https://m.media-amazon.com/images/I/41zDXFipUm...
153    https://m.media-amazon.com/images/I/71HxhWJlza...
154    https://m.media-amazon.com/images/I/71NNd7bV8P...
Name: product_url, Length: 155, dtype: object

Function to check if url contains a image or not

In [4]:
import requests
def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   r = requests.head(image_url)
   if r.headers["content-type"] in image_formats:
      # print(r.headers["content-type"])
      return True
   return False

In [5]:
is_url_image('https://m.media-amazon.com/images/I/71vdb0uL3xL._SX695_.jpg')

False

In [5]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (119, 172, 152)
actual_name, closest_name = get_colour_name(requested_colour)

In [12]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[1])

gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined
gone from here name 'get_colour_name' is not defined


Empty String variable to store results

In [30]:
txt = ""

Function to extract information from image

In [39]:
def extract_info(uri):
    global txt
    try:
        if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            #For retriving Brand
            # res_logo = client.logo_detection(img)
            # logos = res_logo.logo_annotations
            # logo = [log.description for log in logos]
            # txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
            
            #For retriving Labels
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            # txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)}, "
            
            # #For retriving Text
            # res_text = client.document_text_detection(img)
            # texts = res_text.text_annotations
            # text = [t.description for t in texts]
            # txt = txt + f"Text: None, " if len(text)==0 else txt + f"Text: {text[0]}, "
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            
            txt = list(set(obj).intersection(label)) if set(obj).intersection(label) else label[:2]
            
            return str(txt)
        else:
            txt = "Please enter a valid URL that contains Image"
    except Exception as e:
        txt = f"Please Enter a Valid URL of Image"
    return str(txt)

In [40]:
du = data_url[:10]
du

0    https://m.media-amazon.com/images/I/71xWtLhH2G...
1    https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2    https://m.media-amazon.com/images/I/71YG7EClYf...
3    https://m.media-amazon.com/images/I/81ATe15IyH...
4    https://m.media-amazon.com/images/I/61OBcY37KX...
5    https://m.media-amazon.com/images/I/710+f7XX2F...
6    https://m.media-amazon.com/images/I/71u2XOFXAI...
7    https://m.media-amazon.com/images/I/61U0BBQYhr...
8    https://m.media-amazon.com/images/I/71IQc8cAIQ...
9    https://m.media-amazon.com/images/I/71fKZrHSO9...
Name: product_url, dtype: object

In [41]:
d = data[:10]
d

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...
5,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...
6,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...
7,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...
8,ADIDAS Mens Regular Fit M Sereno Bos T2 Jerseys,https://m.media-amazon.com/images/I/71IQc8cAIQ...
9,adidas Men's Regular Fit M D2M Polo,https://m.media-amazon.com/images/I/71fKZrHSO9...


Creating new dataframe to save results

In [42]:
result = d
result.head()

,product_name,product_url
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...


Iterating through function to extract features of image and adding it to result dataframe

In [43]:
for i in range(len(du)):
    print(i)
    extract_info(du[i])
    result.loc[i, "objects_extracted"] = txt
    

0


C:\Users\Microsoft\AppData\Local\Temp\ipykernel_11244\4098288774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.loc[i, "objects_extracted"] = txt


1
2
3
4


ValueError: Must have equal len keys and value when setting with an iterable

Resultant Dataframe

In [35]:
result

,product_name,product_url,objects_extracted
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,None
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,None
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,None
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,None
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,None
5,Bata Mens MaxUniform Dress Shoe,https://m.media-amazon.com/images/I/710+f7XX2F...,None
6,SPARX Mens Sx0675g Walking Shoe,https://m.media-amazon.com/images/I/71u2XOFXAI...,None
7,Campus Stack Sneakers for Men,https://m.media-amazon.com/images/I/61U0BBQYhr...,None
8,ADIDAS Mens Regular Fit M Sereno Bos T2 Jerseys,https://m.media-amazon.com/images/I/71IQc8cAIQ...,None
9,adidas Men's Regular Fit M D2M Polo,https://m.media-amazon.com/images/I/71fKZrHSO9...,None


Saving Dataframe to CSV file

In [4]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

NameError: name 'result' is not defined

In [5]:
nnn = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
nnn.shape

(0, 3)

In [14]:
nnn.to_csv("gs://bucket-shreyash/Product_Data/Product_D.csv",index=False)

In [3]:
df= pd.read_csv("gs://bucket-shreyash/Product_Data/Product_D.csv")
df

,product_name,product_url,object,label
0,Sparx Men's Sc0162g Sneakers,https://m.media-amazon.com/images/I/71xWtLhH2G...,['Shoe'],"['Footwear', 'Shoe', 'Product', 'Walking shoe'..."
1,Puma Womens Kendall WNS Running Shoe,https://m.media-amazon.com/images/I/61C+Y9zoAQ...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
2,Nivia Men's Combat 2.0 Basketball Shoes,https://m.media-amazon.com/images/I/71YG7EClYf...,['Shoe'],"['Footwear', 'Shoe', 'Outdoor shoe', 'Walking ..."
3,adidas Mens Amalgo M Running Shoe,https://m.media-amazon.com/images/I/81ATe15IyH...,['Shoe'],"['Footwear', 'Shoe', 'Walking shoe', 'Outdoor ..."
4,Red Tape Casual Sneaker Shoes for Men,https://m.media-amazon.com/images/I/61OBcY37KX...,['Shoe'],"['Footwear', 'Outdoor shoe', 'Walking shoe', '..."
...,...,...,...,...
182,"Fresh Banana Robusta, 1kg",https://m.media-amazon.com/images/I/51ebZJ+DR4...,['Banana'],"['Food', 'Banana', 'Plant', 'Cooking plantain'..."
183,Bajaj Frore Neo Table Fan 400 MM,https://m.media-amazon.com/images/I/61uCb3XScb...,['Mechanical fan'],"['Plant', 'Electric fan', 'Home appliance', 'T..."
184,"Fresh Water Melon - Kiran, 1 Pc, 2 kg, fresh",https://m.media-amazon.com/images/I/71ZaS7W9dY...,"['Watermelon', 'Fruit']","['Food', 'Fruit', 'Plant', 'Watermelon', 'Citr..."
185,"Fresh Avocado, 1 Pc pack",https://m.media-amazon.com/images/I/71cs5TNn-L...,['Fruit'],"['Food', 'Fruit', 'Natural foods', 'Staple foo..."


In [17]:
print(df.to_string())

                                                                                                        product_name                                                                        product_url                              objects_extracted
0                                                                                       Sparx Men's Sc0162g Sneakers                        https://m.media-amazon.com/images/I/71xWtLhH2GL._SX695_.jpg                                       ['Shoe']
1                                                                               Puma Womens Kendall WNS Running Shoe                        https://m.media-amazon.com/images/I/61C+Y9zoAQL._SY695_.jpg                                       ['Shoe']
2                                                                            Nivia Men's Combat 2.0 Basketball Shoes                        https://m.media-amazon.com/images/I/71YG7EClYfL._SX679_.jpg                                       {'Shoe'}
3           

In [12]:
query='shoe'

In [16]:
if df['product_name'].str.lower().str.contains(query).any():
    name = df[df['product_name'].str.lower().str.contains(query)]['product_name'].tolist()
    print(name)
    link = df[df['product_name'].str.lower().str.contains(query)]['product_url'].tolist()

['Puma Womens Kendall WNS Running Shoe', "Nivia Men's Combat 2.0 Basketball Shoes", 'adidas Mens Amalgo M Running Shoe', 'Red Tape Casual Sneaker Shoes for Men', 'Bata Mens MaxUniform Dress Shoe', 'SPARX Mens Sx0675g Walking Shoe']


In [9]:

namee = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes').any()]['product_name'].tolist()
link = a[a['product_name'].map(lambda x: x.lower()).str.contains('shoes')]['product_url'].tolist()

In [10]:
namee

['Leader Spyder 27.5T MTB Cycle/Bike',
 'Leader Scout MTB 26T Mountain Bicycle/Bike',
 'Amitasha Pull Back Motorbike High-Speed Racing Bike',
 'Baybee R7 Battery Operated Bike for Kids',
 'Nexus Product Bike for Kids Toy R3 Bike',
 'PLUSPOINT Diecast Motorcycle Toy Ducati Bike']

In [25]:
d = a['product_name'].map(lambda x: x.lower())

In [26]:
d

0                           sparx men's sc0162g sneakers
1                   puma womens kendall wns running shoe
2                nivia men's combat 2.0 basketball shoes
3                      adidas mens amalgo m running shoe
4                  red tape casual sneaker shoes for men
                             ...                        
152    gormery 1/12 scale model compatible with kawasaki
153              nexus product bike for kids toy r3 bike
154         pluspoint diecast motorcycle toy ducati bike
155                 solakaka sm600 wireless gaming mouse
156    zebronics zeb-blaze rgb gaming mouse pad with ...
Name: product_name, Length: 157, dtype: object

In [78]:
ob = nnn['objects_extracted']

In [91]:
import ast

e = {'Television'}
for i in ob:
    i = set(ast.literal_eval(i))
    if len(e.intersection(i)) !=0:
        print(i)


{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}
{'Television'}


Uploading csv file to Cloud Stroage

In [2]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
